# 04 - Evaluación del Modelo en el Conjunto de Test

**Materia:** Redes Neuronales Profundas — UTN FRM

**Objetivo:** Evaluar el modelo fine-tuneado sobre el conjunto de test, calcular métricas de clasificación y realizar pruebas cualitativas.

---

## Métricas de Evaluación

- **Accuracy:** Proporción de predicciones correctas sobre el total.
- **Precision:** De los predichos como positivos, ¿cuántos realmente lo son?
- **Recall:** De los positivos reales, ¿cuántos detectó el modelo?
- **F1-Score:** Media armónica de Precision y Recall.
- **MCC (Matthews Correlation Coefficient):** Métrica robusta que considera las 4 celdas de la matriz de confusión. Rango -1 a +1.
- **Matriz de Confusión:** Tabla con TP, TN, FP, FN.

## 1. Importación de Librerías

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Configuración

In [ ]:
TENSORS_DIR = "../data/tensors/"
MODEL_DIR = "../data/model_save/"
RESULTS_DIR = "../data/results/"
BATCH_SIZE = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")
if torch.cuda.is_available(): print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Carga del Modelo Fine-Tuneado

In [ ]:
print(f"Cargando modelo desde {MODEL_DIR}...")
model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
model.to(device)
print(f"Modelo cargado: {type(model).__name__}")

## 4. Carga del Conjunto de Test

In [ ]:
test_dataset = torch.load(os.path.join(TENSORS_DIR, "test_dataset.pt"), weights_only=False)
print(f"Test: {len(test_dataset):,} muestras")

test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=BATCH_SIZE)

## 5. Generación de Predicciones

Recorremos todos los batches en modo evaluación (`model.eval()`) sin gradientes (`torch.no_grad()`).

In [ ]:
print(f"Prediciendo para {len(test_dataset):,} muestras...")
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, return_dict=True)

    predictions.append(result.logits.detach().cpu().numpy())
    true_labels.append(b_labels.to('cpu').numpy())

flat_predictions = np.argmax(np.concatenate(predictions, axis=0), axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)
print("Predicción completa.")

## 6. Cálculo de Métricas

In [ ]:
acc = accuracy_score(flat_true_labels, flat_predictions)
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print(f"Accuracy: {acc:.4f}")
print(f"MCC: {mcc:.4f}")

target_names = ['Negativa', 'Positiva']
report = classification_report(flat_true_labels, flat_predictions, target_names=target_names)
print(f"\nClassification Report:\n{report}")

## 7. Matriz de Confusión

Visualiza TN (verdaderos negativos), FP (falsos positivos), FN (falsos negativos) y TP (verdaderos positivos).

In [ ]:
cm = confusion_matrix(flat_true_labels, flat_predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Negativa', 'Positiva'], yticklabels=['Negativa', 'Positiva'])
plt.ylabel('Etiqueta Real')
plt.xlabel('Predicción')
plt.title('Matriz de Confusión - Steam Review Classifier')
plt.tight_layout()

os.makedirs(RESULTS_DIR, exist_ok=True)
plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrix.png"), dpi=150)
plt.show()

## 8. Curvas de Pérdida

In [ ]:
stats_path = os.path.join(MODEL_DIR, "training_stats.csv")
if os.path.exists(stats_path):
    df_stats = pd.read_csv(stats_path)
    sns.set(style='darkgrid'); sns.set(font_scale=1.5)
    plt.figure(figsize=(12, 6))
    plt.plot(df_stats['epoch'], df_stats['Training Loss'], 'b-o', label='Entrenamiento')
    plt.plot(df_stats['epoch'], df_stats['Valid. Loss'], 'g-o', label='Validación')
    plt.title('Pérdida de Entrenamiento y Validación')
    plt.xlabel('Época'); plt.ylabel('Pérdida'); plt.legend()
    plt.xticks(df_stats['epoch']); plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'training_loss.png'), dpi=150)
    plt.show()

## 9. Pruebas Cualitativas

Probamos con reseñas de ejemplo para verificar el comportamiento del modelo más allá de las métricas numéricas.

In [ ]:
examples = [
    "This game is absolutely amazing, I love the graphics and gameplay!",
    "Terrible game, crashes every 5 minutes. Do not buy this garbage.",
    "It's okay, nothing special but not bad either. Average game.",
    "Best game I have ever played. Hundreds of hours of fun!",
    "Waste of money. The developers abandoned this project.",
    "Really fun with friends, great multiplayer experience.",
]

print("=" * 50)
print("  PRUEBAS CUALITATIVAS")
print("=" * 50)

model.eval()
for text in examples:
    encoded = tokenizer(text, add_special_tokens=True, max_length=128, padding='max_length',
                        truncation=True, return_attention_mask=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        result = model(input_ids, attention_mask=attention_mask, return_dict=True)
    probs = torch.softmax(result.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred].item()
    label = 'POSITIVA' if pred == 1 else 'NEGATIVA'
    print(f'\n  [{label}] (confianza: {confidence:.2%})')
    print(f'  "{text}"')

## 10. Guardado de Métricas

In [ ]:
with open(os.path.join(RESULTS_DIR, 'metrics.txt'), 'w') as f:
    f.write(f'Accuracy: {acc:.4f}\n')
    f.write(f'MCC: {mcc:.4f}\n\n')
    f.write(f'Classification Report:\n{report}\n')
    f.write(f'Confusion Matrix:\n{cm}\n')
print(f"Resultados guardados en: {RESULTS_DIR}")

## Resumen

### Resultados en Test:

| Métrica | Valor |
|---|---|
| **Accuracy** | 90.56% |
| **MCC** | 0.8112 |
| **Precision (Neg/Pos)** | 0.91 / 0.90 |
| **Recall (Neg/Pos)** | 0.90 / 0.91 |
| **F1-Score (macro)** | 0.91 |

El modelo alcanza 90.56% de accuracy y MCC de 0.81, con métricas equilibradas entre clases. Las pruebas cualitativas confirman clasificación correcta.

**Siguiente paso:** Exportar el modelo para producción.